In [2]:
import torch,random
import matplotlib
% matplotlib inline
import matplotlib.pyplot as plt

def acc(src,L_used,L):
    # src: result directory
    # L_used: how many ensemble results are randomly picked
    # L : total ensembled size
    
    # loading dataset info
    dsets = torch.load(src + 'dsets.pth')
    N = len(dsets['val'])
    imgCat = dsets['val'].classDict

    idx = [i for i in range(L)]
    random.shuffle(idx)
    idx = idx[:L_used]
    
    # loading feature vectors
    R = [torch.load(src+str(d)+'valFvecs.pth') for d in idx]
    R = torch.stack(R,2)
    print(R.size())
    
    # calculating the similarity matrix
    with torch.cuda.device(2):
        R = R.permute(0,2,1).contiguous().view(R.size(0),-1).cuda()

        D = R.mm(R.t())
        D[torch.eye(N).byte()]=-1

        A,B = D.sort(1,descending=True)
    
    # calculating the recall accuracy
    T1,T2 = 0,0
    for i in range(N):
        score,idx = A[i,:],B[i,:]

        if imgCat[idx[0].item()]==imgCat[i]: 
            T1+=1

        for j in range(2):
            if imgCat[idx[j]]==imgCat[i]: 
                T2+=1
                break

    print('R@1:{:.4f} R@2:{:.4f}'.format(T1/N,T2/N),end='\r')
    return T1/N,T2/N